In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

#### BIID

In [3]:
path = "Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Data Modeling Clean/"

In [4]:
biid = pd.read_adls(path + "/_temp/" + "busishell_v31" + "_all.parquet", reader = pd.read_parquet)
# biid = pd.read_adls(path + "/_temp/" + "busishell_v31_all_ver_src" + ".parquet", reader = pd.read_parquet)
print(biid.shape)

(7971880, 183)


#### Profiling Analysis

In [18]:
# use input_df
input_df = pd.read_parquet("./_temp/profile_seleid.parquet")
print(input_df.shape)
input_df.columns = ["accountnumber", "ecl_seleid"]

(7964072, 2)


In [19]:
biid = biid.merge(input_df, left_on = "account", right_on = "accountnumber")
print(biid.shape)

(7964072, 185)


In [20]:
biid[(biid["id_seleid"]!= biid["ecl_seleid"])].shape

(105722, 185)

In [21]:
fmt = make_format(cuts = [-np.inf, 0, np.inf])
biid["ecl_seleid"] = biid["ecl_seleid"].astype(float)
biid["id_seleid"] = biid["id_seleid"].astype(float)
freq(biid["ecl_seleid"], biid["id_seleid"], cross = False, observed = True, format = [fmt, fmt])

Count       Pct  Cuml Count  Cuml Pct
ecl_seleid id_seleid                                           
<= 0       <= 0         72714.0  0.009130     72714.0  0.009130
           1+           96958.0  0.012174    169672.0  0.021305
           Missing        367.0  0.000046    170039.0  0.021351
1+         <= 0           574.0  0.000072    170613.0  0.021423
           1+         7793354.0  0.978564   7963967.0  0.999987
           Missing        105.0  0.000013   7964072.0  1.000000

In [22]:
biid = biid[(biid["id_seleid"]!=0) & (biid.id_seleid == biid.ecl_seleid)]
print(biid.shape)
biid["count"] = 1
biid_roll =biid.groupby(by = ["id_seleid", "history_datetime"])['count'].sum().reset_index(name='count')
biid_roll["count"].describe()

(7785636, 185)


count    7.030200e+06
mean     1.107456e+00
std      8.555810e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      4.440000e+02
Name: count, dtype: float64

In [23]:
fmt = make_format(cuts = [-np.inf, -1, 0, 1, 2, 3, 4, 5, 10, 50, 100, np.inf])
freq("count", df = biid_roll, format = fmt, observed = True)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,6506877.0,0.925561,6506877.0,0.925561
2,459527.0,0.065365,6966404.0,0.990925
3,35341.0,0.005027,7001745.0,0.995952
4,11040.0,0.001570,7012785.0,0.997523
5,4242.0,0.000603,7017027.0,0.998126
6-10,7335.0,0.001043,7024362.0,0.999170
11-50,5706.0,0.000812,7030068.0,0.999981
51-100,90.0,0.000013,7030158.0,0.999994
101+,42.0,0.000006,7030200.0,1.000000


In [24]:
print(biid.shape)

(7785636, 186)


In [25]:
biid = biid.drop_duplicates(subset = ["id_seleid", "history_datetime"], ignore_index = True)
print(biid.shape)

(7030200, 186)


In [26]:
biid_roll =biid.groupby(by = ["id_seleid"])['count'].sum().reset_index(name='count')
biid_roll["count"].describe()

count    1.311745e+06
mean     5.359426e+00
std      6.676770e+00
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      9.000000e+00
max      1.950000e+02
Name: count, dtype: float64

In [27]:
freq("count", df = biid_roll, format = fmt, observed = True)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,553706.0,0.422114,553706.0,0.422114
2,234564.0,0.178818,788270.0,0.600932
3,57023.0,0.043471,845293.0,0.644403
4,29306.0,0.022341,874599.0,0.666745
5,21105.0,0.016089,895704.0,0.682834
6-10,212184.0,0.161757,1107888.0,0.844591
11-50,203843.0,0.155398,1311731.0,0.999989
51-100,12.0,0.000009,1311743.0,0.999998
101+,2.0,0.000002,1311745.0,1.000000


In [28]:
# biid_roll[biid_roll["count"] == 195]

In [29]:
biid_roll["id_seleid"] = biid_roll["id_seleid"].astype(float)
biid["id_seleid"] = biid["id_seleid"].astype(float)
biid = biid.merge(biid_roll, on = "id_seleid")
print(biid.shape)

(7030200, 187)


In [35]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, np.inf], exceptions = [-1])
freq(biid.assoc_count.astype(float), format = [fmt])

,Count,Pct,Cuml Count,Cuml Pct
assoc_count,,,,
<= 0,908397.0,0.129214,908397.0,0.129214
1,2285335.0,0.325074,3193732.0,0.454288
2,1230520.0,0.175033,4424252.0,0.629321
3,517684.0,0.073637,4941936.0,0.702958
4,272942.0,0.038824,5214878.0,0.741782
5,156412.0,0.022249,5371290.0,0.764031
6,99927.0,0.014214,5471217.0,0.778245
7,65368.0,0.009298,5536585.0,0.787543
8,46351.0,0.006593,5582936.0,0.794136


In [36]:
fmt = make_format(cuts = [-np.inf, 0, 500_000, 1_000_000, 2_000_000, 3_000_000, 4_000_000, 5_000_000, np.inf], exceptions = [-1])
freq(biid.fin_reported_sales.astype(float), format = [fmt], )

,Count,Pct,Cuml Count,Cuml Pct
fin_reported_sales,,,,
<= 0,271695.0,0.038647,271695.0,0.038647
1-500000,1865246.0,0.265319,2136941.0,0.303966
500001-1000000,475234.0,0.067599,2612175.0,0.371565
1000001-2000000,465438.0,0.066206,3077613.0,0.437770
2000001-3000000,212676.0,0.030252,3290289.0,0.468022
3000001-4000000,122136.0,0.017373,3412425.0,0.485395
4000001-5000000,78175.0,0.011120,3490600.0,0.496515
5000001+,316008.0,0.044950,3806608.0,0.541465
-1,3223592.0,0.458535,7030200.0,1.000000


In [ ]:
def left_closed_labels(b, include_max=False, fmt=".3f", step=1):
    lbs = []
    for i in range(len(b) - 1):
        if b[i] == (b[i + 1] - step):
            lbs.append(f"{b[i]:{fmt}}")
        else:
            lbs.append(f"{b[i]:{fmt}}-{b[i+1] - step:{fmt}}")
    if not include_max:
        lbs[-1] = f"{b[-2]:{fmt}}+"
    return lbs

In [ ]:
biid = biid.apply(pd.to_numeric, errors='ignore')

In [ ]:
## create flag
biid["many_inquiry"] = np.where(biid['count_y'].astype(int) >25, 1, 0)
freq(biid["many_inquiry"])

,Count,Pct,Cuml Count,Cuml Pct
many_inquiry,,,,
0,6945091.0,0.987894,6945091.0,0.987894
1,85109.0,0.012106,7030200.0,1.000000


In [ ]:
freq(biid.sbfe_worst_perf)

,Count,Pct,Cuml Count,Cuml Pct
sbfe_worst_perf,,,,
-99,314580.0,0.044747,314580.0,0.044747
-97,606.0,0.000086,315186.0,0.044833
0,3385014.0,0.481496,3700200.0,0.526329
1,47420.0,0.006745,3747620.0,0.533074
2,1601092.0,0.227745,5348712.0,0.760819
3,550497.0,0.078305,5899209.0,0.839124
4,182037.0,0.025894,6081246.0,0.865017
5,106329.0,0.015125,6187575.0,0.880142
6,87618.0,0.012463,6275193.0,0.892605


In [ ]:
biv_all = {}
biv_client = {}
for s in keep:
    if pd.api.types.is_string_dtype(biid[s].dtype):
        temp_biv = bivariate(biid[s], biid["many_inquiry"], observed = True).fillna("n/a")
    else:
        if biid[s].nunique() <=15:
            temp_biv = bivariate(biid[s], biid["many_inquiry"], observed = True).fillna("n/a")
        else:
            brks = (
                    biid[s].pipe(lambda x: x[x.gt(-1)])
                    .quantile([0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99])
                )
            dup = [x+1 for x in brks if brks.tolist().count(x)>1]
            brks = brks.tolist() + dup
            brks = np.unique(brks)

            if brks.max() == 0:
                brks = np.append(brks, 1)
            brks.sort()
            brks = np.append(brks, np.inf) 

            labs = left_closed_labels(brks, fmt=".0f")
            fmt = make_format(
                cuts=brks, labels=labs, right=False, exceptions=[-99_999, -99_998, -99_997, -1, -99, -98, -97]
            )
            temp_biv = bivariate(biid[s], biid["many_inquiry"], format=fmt, observed = True).fillna("n/a")
            
    biv_all[s] = temp_biv
    
wb = TableWriter(filename = "./_temp/temp_bus_shell_biv.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in biv_all.keys():
    wb.write_table(
        biv_all[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

In [ ]:
pd.set_option('display.max_columns', None)
biid.head()

,account,seq,id_powid,id_proxid,id_seleid,id_orgid,id_ultid,id_seleid_change_flag,id_seleid_change_code,id_weight,id_score,id_category,id_status,id_truebiz,history_date,history_datetime,in_bus_state,pop_bus_name,pop_bus_addr,pop_bus_city,pop_bus_state,pop_bus_zip,pop_bus_fein,pop_bus_phone,pop_rep1_first,pop_rep1_last,pop_rep1_addr,pop_rep1_ssn,ver_src_id_firstseen,ver_src_id_lastseen,ver_src_id_mth_since_fs,ver_src_id_mth_since_ls,ver_src_id_count,ver_src_firstseen,ver_src_lastseen,ver_src_mth_since_fs,ver_src_mth_since_ls,ver_src_bureau_mth_since_fs,ver_src_count,ver_src_irsnonprofit,ver_src_id_nonderog_count,ver_src_id_nonderog_count3,ver_src_id_nonderog_count6,ver_src_id_nonderog_count12,ver_src_id_nonderog_count24,ver_src_id_nonderog_count36,ver_src_id_nonderog_count60,ver_src_id_activity03,ver_src_id_activity06,ver_src_id_activity12,ver_src_id_list,ver_src_id_firstseen_list,ver_src_id_lastseen_list,ver_src_list,ver_name_src_count,ver_addr_src_count,ver_fein_src_count,ver_phn_src_id_count,ver_phn_src_count,addr_input_miskey,addr_input_valid,addr_input_not_most_recent,addr_input_type_advo,addr_reported_type,addr_input_zipcode_type,addr_input_zipcode_ver,addr_input_assessed_value,addr_input_lot_size,addr_input_lres,addr_best_src_count,phn_input_miskey,phn_input_mismatch,phn_input_contradictory,phn_input_disconnected,phn_input_valid,phn_input_type,phn_input_residential,phn_input_problems,phn_input_distance_addr,fein_input_contradictory_id,fein_input_contradictory_in,cons_record_match_name,cons_record_match_addr,cons_record_match_phone,tin_match_cons_name,tin_match_cons_w_close_addr,tin_match_cons_associate,tin_match_cons_hhmember,e2b_rep1_name_on_file,e2b_rep1_addr_on_file,e2b_rep1_ssn_on_file,e2b_rep1_busheader_phone,e2b_rep1_busheader_ssn,e2b_rep1_idsearch_name,e2b_rep1_idsearch_addr,e2b_rep1_idsearch_phone,e2b_rep1_idsearch_ssn,e2b_rep1_consheader_addr,e2b_rep1_consheader_ssn,e2b_rep1_paw_match,e2b_rep1_distance_addr,e2b_rep1_match_bus_file_addr,e2b_rep1_on_inquiry,assoc_count,assoc_business_total,assoc_felony_total,assoc_felony_count,assoc_bankruptcy_count12,assoc_lien_count,assoc_judg_total,assoc_judg_count,assoc_paw_count,assoc_city_match_count,assoc_county_match_count,vel_addr_per_id_curr,sos_inc_filing_count,sos_inc_filing_firstseen,sos_inc_filing_lastseen,sos_state_input_match,sos_state_foreign_reported,sos_foreign_count,sos_foreign_state_count,sos_standing,sos_ever_defunct,sos_agent_change_lastseen,lien_filed_count,lien_filed_firstseen,lien_filed_ot_ct,ucc_count,ucc_firstseen,ucc_lastseen,ucc_active_count,ucc_terminated_count,ucc_other_count,prop_assessed_value_total,ind_naics_best,ind_sic_best,empl_ct_best,empl_ct_best_range,fin_reported_sales,fin_reported_earnings,inq_total_count,inq_total_count03,inq_total_count06,inq_total_count12,inq_total_count24,inq_credit_count,inq_credit_count03,inq_credit_count06,inq_credit_count12,inq_credit_count24,inq_highrisk_count,inq_highrisk_count03,inq_highrisk_count06,inq_highrisk_count12,inq_highrisk_count24,inq_oth_count,inq_oth_count03,inq_oth_count06,inq_oth_count12,inq_oth_count24,inq_consumer_addr,inq_consumer_phone,sbfe_source_index,sbfe_mth_since_fs,sbfe_acct_cnt,sbfe_acct_opened_cnt03,sbfe_acct_opened_cnt06,sbfe_acct_opened_cnt12,sbfe_acct_opened_cnt24,sbfe_acct_opened_cnt36,sbfe_acct_opened_cnt60,sbfe_acct_opened_cnt84,sbfe_open_cnt,sbfe_worst_perf00,sbfe_worst_perf03,sbfe_worst_perf06,sbfe_worst_perf12,sbfe_worst_perf24,sbfe_worst_perf36,sbfe_worst_perf60,sbfe_worst_perf84,sbfe_worst_perf,accountnumber,ecl_seleid,count_x,count_y,many_inquiry
0,AAA000000102409565,0,138813849069,138813849069,1.388138e+11,138813849069,138813849069,0,0,57,0,5 TRUSTED SOURCE,GOLD,0,202201,20220119,CA,1,1,1,1,1,1,0,0,0,0,0,0,0,-1,-1,-1,0,0,-1,-1,-1,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,None,None,None,None,0,0,0,-1,-1,-1,1,-1,A,U,4,-1,197240,7841,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,

In [ ]:
keep = ['id_status',
 'id_truebiz',
 'pop_bus_name',
 'pop_bus_addr',
 'pop_bus_city',
 'pop_bus_state',
 'pop_bus_zip',
 'pop_bus_fein',
 'pop_bus_phone',
 'pop_rep1_first',
 'pop_rep1_last',
 'pop_rep1_addr',
 'pop_rep1_ssn',
 'ver_src_id_mth_since_fs',
 'ver_src_id_mth_since_ls',
 'ver_src_id_count',
 'ver_src_mth_since_fs',
 'ver_src_mth_since_ls',
 'ver_src_bureau_mth_since_fs',
 'ver_src_count',
 'ver_src_irsnonprofit',
 'ver_src_id_nonderog_count',
 'ver_src_id_nonderog_count3',
 'ver_src_id_nonderog_count6',
 'ver_src_id_nonderog_count12',
 'ver_src_id_nonderog_count24',
 'ver_src_id_nonderog_count36',
 'ver_src_id_nonderog_count60',
 'ver_src_id_activity03',
 'ver_src_id_activity06',
 'ver_src_id_activity12',
 'ver_name_src_count',
 'ver_addr_src_count',
 'ver_fein_src_count',
 'ver_phn_src_id_count',
 'ver_phn_src_count',
 'addr_input_miskey',
 'addr_input_valid',
 'addr_input_not_most_recent',
 'addr_input_type_advo',
 'addr_reported_type',
 'addr_input_zipcode_type',
 'addr_input_zipcode_ver',
 'addr_input_assessed_value',
 'addr_input_lot_size',
 'addr_input_lres',
 'addr_best_src_count',
 'phn_input_miskey',
 'phn_input_mismatch',
 'phn_input_contradictory',
 'phn_input_disconnected',
 'phn_input_valid',
 'phn_input_type',
 'phn_input_residential',
 'phn_input_problems',
 'phn_input_distance_addr',
 'fein_input_contradictory_id',
 'fein_input_contradictory_in',
 'cons_record_match_name',
 'cons_record_match_addr',
 'cons_record_match_phone',
 'tin_match_cons_name',
 'tin_match_cons_w_close_addr',
 'tin_match_cons_associate',
 'tin_match_cons_hhmember',
 'e2b_rep1_name_on_file',
 'e2b_rep1_addr_on_file',
 'e2b_rep1_ssn_on_file',
 'e2b_rep1_busheader_phone',
 'e2b_rep1_busheader_ssn',
 'e2b_rep1_idsearch_name',
 'e2b_rep1_idsearch_addr',
 'e2b_rep1_idsearch_phone',
 'e2b_rep1_idsearch_ssn',
 'e2b_rep1_consheader_addr',
 'e2b_rep1_consheader_ssn',
 'e2b_rep1_paw_match',
 'e2b_rep1_distance_addr',
 'e2b_rep1_match_bus_file_addr',
 'e2b_rep1_on_inquiry',
 'assoc_count',
 'assoc_business_total',
 'assoc_felony_total',
 'assoc_felony_count',
 'assoc_bankruptcy_count12',
 'assoc_lien_count',
 'assoc_judg_total',
 'assoc_judg_count',
 'assoc_paw_count',
 'assoc_city_match_count',
 'assoc_county_match_count',
 'vel_addr_per_id_curr',
 'sos_inc_filing_count',
 'sos_inc_filing_firstseen',
 'sos_inc_filing_lastseen',
 'sos_state_input_match',
 'sos_state_foreign_reported',
 'sos_foreign_count',
 'sos_foreign_state_count',
 'sos_standing',
 'sos_ever_defunct',
 'sos_agent_change_lastseen',
 'lien_filed_count',
 'lien_filed_firstseen',
 'lien_filed_ot_ct',
 'ucc_count',
 'ucc_firstseen',
 'ucc_lastseen',
 'ucc_active_count',
 'ucc_terminated_count',
 'ucc_other_count',
 'prop_assessed_value_total',
 
 'empl_ct_best',
 'empl_ct_best_range',
 'fin_reported_sales',
 'fin_reported_earnings',
 'inq_total_count',
 'inq_total_count03',
 'inq_total_count06',
 'inq_total_count12',
 'inq_total_count24',
 'inq_credit_count',
 'inq_credit_count03',
 'inq_credit_count06',
 'inq_credit_count12',
 'inq_credit_count24',
 'inq_highrisk_count',
 'inq_highrisk_count03',
 'inq_highrisk_count06',
 'inq_highrisk_count12',
 'inq_highrisk_count24',
 'inq_oth_count',
 'inq_oth_count03',
 'inq_oth_count06',
 'inq_oth_count12',
 'inq_oth_count24',
 'inq_consumer_addr',
 'inq_consumer_phone',
 'sbfe_source_index',
 'sbfe_mth_since_fs',
 'sbfe_acct_cnt',
 'sbfe_acct_opened_cnt03',
 'sbfe_acct_opened_cnt06',
 'sbfe_acct_opened_cnt12',
 'sbfe_acct_opened_cnt24',
 'sbfe_acct_opened_cnt36',
 'sbfe_acct_opened_cnt60',
 'sbfe_acct_opened_cnt84',
 'sbfe_open_cnt',
 'sbfe_worst_perf00',
 'sbfe_worst_perf03',
 'sbfe_worst_perf06',
 'sbfe_worst_perf12',
 'sbfe_worst_perf24',
 'sbfe_worst_perf36',
 'sbfe_worst_perf60',
 'sbfe_worst_perf84',
 'sbfe_worst_perf',]

#### Source Analysis

In [ ]:
# keep = ['account', 'id_seleid', 'id_weight', 'id_score', 'id_status', 'id_truebiz', "history_datetime", 'sbfe_source_index',
# 'ver_src_id_list', 'ver_src_list', 'ver_src_id_count', 'ver_src_count', 
# 'ver_src_id_mth_since_fs', 'ver_src_id_mth_since_ls', 'ver_src_mth_since_fs', 'ver_src_mth_since_ls',]

In [15]:
biid.id_seleid = biid.id_seleid.astype(float)
fmt = make_format(cuts = [-np.inf, 0, np.inf])
freq(biid.id_seleid, format = [fmt])

,Count,Pct,Cuml Count,Cuml Pct
id_seleid,,,,
<= 0,73288.0,0.009202,73288.0,0.009202
1+,7890312.0,0.990738,7963600.0,0.999941
Missing,472.0,0.000059,7964072.0,1.000000


In [6]:
biid = biid[(biid["id_seleid"].astype(float)!=0) & (biid.id_seleid == biid.ecl_seleid)]
print(biid.shape)
biid = biid.drop_duplicates(subset = ["id_seleid", "history_datetime"], ignore_index = True)
print(biid.shape)
# biid = biid[keep]
print(biid.shape)

(7785636, 27)
(7030200, 27)
(7030200, 27)


In [7]:
## merge in input data:
input_df = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Data Modeling Clean/final_filtered_remove_dt_for_dt_modeling.csv", 
             reader = pd.read_csv, dtype = str)
print(input_df.shape)

(7971880, 13)


In [8]:
input_df.head(1)

,AccountNumber,CompanyName,AlternateCompanyName,Addr,City,State,Zip,BusinessPhone,TaxIdNumber,HistoryDate,sufficient_input,count_x,count_y
0,AAA000000101961271,![CDATA[1 SUN SOLAR ELECTRIC L L C]]>,NaN,2708 S HIGHLAND DR,LAS VEGAS,NV,89109,NaN,261265932,20211123,0,1,1


In [9]:
biid = biid.merge(input_df, left_on = "account", right_on = "AccountNumber")
biid.shape

(7030200, 40)

In [10]:
biid["CompanyName_flag"] = np.where(biid.CompanyName.isnull() | (biid.CompanyName == ""), 0, 1)
biid["AlternateCompanyName_flag"] = np.where(biid.AlternateCompanyName.isnull() | (biid.AlternateCompanyName == ""), 0, 1)
biid["full_addr"] = biid.Addr + biid.City + biid.State + biid.Zip
biid["addr_flag"] = np.where(biid.full_addr.isnull() | (biid.full_addr == ""), 0, 1)
biid["BusinessPhone_flag"] = np.where(biid.BusinessPhone.isnull() | (biid.BusinessPhone == ""), 0, 1)
biid["TaxIdNumber_flag"] = np.where(biid.TaxIdNumber.isnull() | (biid.TaxIdNumber == ""), 0, 1)

In [22]:
biid["TaxIdNumber_flag"].value_counts()

0    6715319
1     314881
Name: TaxIdNumber_flag, dtype: int64

In [20]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, np.inf], exceptions = [-1])
result = dict()
fields = ['ver_name_src_count', 'ver_altnm_src_count', 'ver_addr_src_count', 'ver_fein_src_count', 'ver_phn_src_id_count', 'ver_phn_src_count']
result["1"] = freq(biid['ver_src_nonderog_count'].astype(float), format = fmt, observed = True)
result["2"] = bivariate(biid['ver_name_src_count'].astype(float), biid["CompanyName_flag"], format = fmt, observed = True)
result["3"] = bivariate(biid['ver_altnm_src_count'].astype(float), biid["AlternateCompanyName_flag"], format = fmt, observed = True)
result["4"] = bivariate(biid['ver_addr_src_count'].astype(float), biid["addr_flag"], format = fmt, observed = True)
result["5"] = bivariate(biid['ver_fein_src_count'].astype(float), biid["TaxIdNumber_flag"], format = fmt, observed = True)
result["6"] = bivariate(biid['ver_phn_src_id_count'].astype(float), biid["BusinessPhone_flag"], format = fmt, observed = True)
result["7"] = bivariate(biid['ver_phn_src_count'].astype(float), biid["BusinessPhone_flag"], format = fmt, observed = True)



for i, x in enumerate(fields):
    print(x)
    result[i] = freq(biid[x].astype(float), format = [fmt], observed = True)
wb = TableWriter(filename = "./_temp/sources.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

ver_name_src_count
ver_altnm_src_count
ver_addr_src_count
ver_fein_src_count
ver_phn_src_id_count
ver_phn_src_count


In [ ]:
biid["ver_src_count"] = biid["ver_src_count"].astype(float)
biid["ver_src_id_count"] = biid["ver_src_id_count"].astype(float)

biid['src_diff'] = np.where((biid["ver_src_count"] == -1) & (biid["ver_src_id_count"] ==-1), -99, 
                           np.where((biid["ver_src_count"] != -1) & (biid["ver_src_id_count"] ==-1), -98, 
                                   np.where((biid["ver_src_count"] == -1) & (biid["ver_src_id_count"] !=-1), -97, 
                                           biid["ver_src_count"] - biid["ver_src_id_count"])))

freq(biid['src_diff'])

,Count,Pct,Cuml Count,Cuml Pct
src_diff,,,,
-98.0,1230919.0,0.175090,1230919.0,0.175090
-1.0,1054516.0,0.149998,2285435.0,0.325088
0.0,2646609.0,0.376463,4932044.0,0.701551
1.0,1910316.0,0.271730,6842360.0,0.973281
2.0,187840.0,0.026719,7030200.0,1.000000


In [ ]:
freq("ver_src_count", df = biid[biid['src_diff'] == -98])

,Count,Pct,Cuml Count,Cuml Pct
ver_src_count,,,,
0.0,478252.0,0.388532,478252.0,0.388532
1.0,674308.0,0.547809,1152560.0,0.936341
2.0,78359.0,0.063659,1230919.0,1.000000


In [ ]:
freq("ver_src_list", df = biid[(biid['src_diff'] == -98) * (biid['ver_src_count'] > 0)])

,Count,Pct,Cuml Count,Cuml Pct
ver_src_list,,,,
GB,117326.0,0.155880,117326.0,0.155880
"GB,PP",38121.0,0.050648,155447.0,0.206528
PP,556982.0,0.740011,712429.0,0.946539
"PP,GB",40238.0,0.053461,752667.0,1.000000


In [ ]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, np.inf], exceptions = [-1])
freq(biid["ver_src_count"].astype(float), format = [fmt], observed = True)

,Count,Pct,Cuml Count,Cuml Pct
ver_src_count,,,,
<= 0,481429.0,0.068480,481429.0,0.068480
1,878694.0,0.124988,1360123.0,0.193469
2,543408.0,0.077296,1903531.0,0.270765
3,539621.0,0.076758,2443152.0,0.347522
4,523014.0,0.074395,2966166.0,0.421918
5,470309.0,0.066898,3436475.0,0.488816
6-10,1855579.0,0.263944,5292054.0,0.752760
11-15,1271468.0,0.180858,6563522.0,0.933618
16-20,443911.0,0.063143,7007433.0,0.996762


In [ ]:
freq(biid["ver_src_id_count"].astype(float), format = [fmt], observed = True)

,Count,Pct,Cuml Count,Cuml Pct
ver_src_id_count,,,,
1,440830.0,0.062705,440830.0,0.062705
2,522118.0,0.074268,962948.0,0.136973
3,546977.0,0.077804,1509925.0,0.214777
4,470862.0,0.066977,1980787.0,0.281754
5,427307.0,0.060782,2408094.0,0.342536
6-10,1627760.0,0.231538,4035854.0,0.574074
11-15,1253378.0,0.178285,5289232.0,0.752359
16-20,484948.0,0.068981,5774180.0,0.821339
21-30,25101.0,0.003570,5799281.0,0.824910


In [ ]:
# if the have SBFE
biid["ver_src_id_SBFE_flag"] = biid['ver_src_id_list'].str.lower().str.contains('bc')
freq(biid["ver_src_id_SBFE_flag"])

,Count,Pct,Cuml Count,Cuml Pct
ver_src_id_SBFE_flag,,,,
False,5799281.0,0.82491,5799281.0,0.82491
Missing,1230919.0,0.17509,7030200.0,1.00000


In [ ]:
biid["ver_src_SBFE_flag"] = biid['ver_src_list'].str.lower().str.contains('bc')
freq(biid["ver_src_SBFE_flag"])

,Count,Pct,Cuml Count,Cuml Pct
ver_src_SBFE_flag,,,,
False,6548771.0,0.93152,6548771.0,0.93152
Missing,481429.0,0.06848,7030200.0,1.00000


In [ ]:
## code works
biid["ver_src_SBFE_flag"] = biid['ver_src_list'].str.lower().str.contains('l2')
freq(biid["ver_src_SBFE_flag"])

,Count,Pct,Cuml Count,Cuml Pct
ver_src_SBFE_flag,,,,
False,5557359.0,0.790498,5557359.0,0.790498
True,991412.0,0.141022,6548771.0,0.931520
Missing,481429.0,0.068480,7030200.0,1.000000


In [ ]:
freq(biid["sbfe_source_index"])

,Count,Pct,Cuml Count,Cuml Pct
sbfe_source_index,,,,
0,70970.0,0.010095,70970.0,0.010095
1,243610.0,0.034652,314580.0,0.044747
2,1159949.0,0.164995,1474529.0,0.209742
3,5555671.0,0.790258,7030200.0,1.000000


In [ ]:
freq(biid["sbfe_source_index"], biid["ver_src_id_count"].astype(float), format = [None, fmt], with_stats = False)

ver_src_id_count                                          \
                              <= 0         1         2         3         4   
                             Count     Count     Count     Count     Count   
sbfe_source_index                                                            
0                              0.0       0.0       0.0       0.0       0.0   
1                              0.0   49830.0   65943.0   50248.0   26759.0   
2                              0.0       0.0       0.0       0.0       0.0   
3                              0.0  391000.0  456175.0  496729.0  444103.0   

                                                                            \
                          5       6-10      11-15     16-20    21-30 31-40   
                      Count      Count      Count     Count    Count Count   
sbfe_source_index                                                            
0                       0.0        0.0        0.0       0.0      0.0   0.0   
1                   16847.0    29513.0     4231.0     233.0      6.0   0.0   
2                       0.0        0.0        0.0       0.0      0.0   0.0   
3                  410460.0  1598247.0  1249147.0  484715.0  25095.0   0.0   

                                                  
                  41-50   51+         -1 Missing  
                  Count Count      Count   Count  
sbfe_source_index                                 
0                   0.0   0.0    70970.0     0.0  
1                   0.0   0.0        0.0     0.0  
2                   0.0   0.0  1159949.0     0.0  
3                   0.0   0.0        0.0     0.0

In [ ]:
freq("ver_src_list", df = biid[(biid['sbfe_source_index'].astype(float).isin([0, 2])) &(biid["ver_src_count"].astype(float) > 0)])

,Count,Pct,Cuml Count,Cuml Pct
ver_src_list,,,,
GB,117326.0,0.155880,117326.0,0.155880
"GB,PP",38121.0,0.050648,155447.0,0.206528
PP,556982.0,0.740011,712429.0,0.946539
"PP,GB",40238.0,0.053461,752667.0,1.000000
